In [1]:
!pip install mesa-geo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=96500b016e495c0341128db2e1111bf9e1158b0313ed43824a6f886d98943323
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [3]:
!ngrok config add-authtoken 2WBVyOjBuQYHAe5T0JdShzDTcsW_2dxZVTUVJ6CeG3qfUApsc

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
!gdown --id "1M842fQpOLQW-f33UerGUlUUbD2nGk84U"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1M842fQpOLQW-f33UerGUlUUbD2nGk84U
To: /content/europe.geojson
100% 1.63M/1.63M [00:00<00:00, 107MB/s]


In [5]:
f = open("europe.geojson", "r")

In [6]:
import mesa_geo as mg
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import mesa
import statistics

In [7]:
europe = gpd.read_file("europe.geojson")
europe.head()

,FID,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry
0,0.0,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."
1,0.0,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.43621 41.02107, 19.45055 41.06000..."
2,0.0,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"MULTIPOLYGON (((45.57305 40.63249, 45.52888 40..."
3,0.0,BK,BA,BIH,70,Bosnia and Herzegovina,5120,3915238,150,39,17.786,44.169,"POLYGON ((17.64984 42.88908, 17.57853 42.94382..."
4,0.0,BU,BG,BGR,100,Bulgaria,11063,7744591,150,151,25.231,42.761,"POLYGON ((27.87917 42.84110, 27.89500 42.80250..."


In [8]:
class HumanAgent(mg.GeoAgent):
  def __init__(self, unique_id, model, geometry, crs, employed = False, startHap = 0.7, resource_consumption = 0.1):
    super().__init__(unique_id, model, geometry, crs)
    self.employed = employed
    self.happiness = startHap
    self.resource_consumption = resource_consumption

  def step(self):
    self.affectEnvironment()
    self.determineHappiness()
    self.seekEmployment()

  # looks for new job if unemployed or unhappy
  def seekEmployment(self):
    if self.happiness < 0.3 or not self.employed:
      openjobs = [job for job in self.model.schedule.agents if isinstance(job, JobAgent) and job.taken == False]
      currentjob = [job for job in self.model.space.get_intersecting_agents(self) if isinstance(job, JobAgent)]
      if openjobs:
        newjob = self.random.choice(openjobs)
        newjob.taken = True
        if currentjob:
          currentjob[0].taken = False
        x,y = newjob.geometry.centroid.coords.xy
        self.geometry = Point(x,y)
        self.employed = True
        self.happiness = 0.6


  #adjusts happiness based on country environment
  def determineHappiness(self):
    curr_country = [country for country in self.model.space.get_intersecting_agents(self) if isinstance(country, CountryAgent)][0]
    if curr_country:
      if curr_country.env_cond < 0.3:
        self.happiness = max(self.happiness-0.1, 0)
      else:
        self.happiness = min(self.happiness + 0.1, 1)
  #affects environment of country agent is in
  def affectEnvironment(self):
    curr_country = [country for country in self.model.space.get_intersecting_agents(self) if isinstance(country, CountryAgent)][0]
    if curr_country:
      curr_country.env_cond = max(self.resource_consumption - self.resource_consumption, 0)


class JobAgent(mg.GeoAgent):
  def __init__(self, unique_id, model, geometry, crs, taken = False):
    super().__init__(unique_id, model, geometry,crs)
    self.taken = taken

class CountryAgent(mg.GeoAgent):
  def __init__(self, unique_id, model, geometry, crs, init_env_cond = 0.5, env_regenr = 0.15):
    super().__init__(unique_id, model,geometry, crs)
    self.env_cond = init_env_cond
    self.env_regenr = env_regenr

  def step(self):
    self.regenResource()
  #regenerates a certain amount of resource per step
  def regenResource(self):
    self.env_cond = min(self.env_cond+self.env_regenr,1)


In [9]:
def avg_happiness(model):
  return statistics.mean([agent.happiness for agent in model.schedule.agents if isinstance(agent, HumanAgent)])

def avg_env_cond(model):
  return statistics.mean([agent.env_cond for agent in model.schedule.agents if isinstance(agent, CountryAgent)])

class EnvModel(mesa.Model):
  geojson_regions = "europe.geojson"
  unique_id = "NAME"
  def __init__(self, agents = 30, totJobs = 20, perc_employed = 0.5, resource_consumption = 0.1, env_regen_rate = 0.15):
    # initialize model parameters
    self.schedule = mesa.time.RandomActivation(self)
    self.space = mg.GeoSpace(warn_crs_conversion = False)
    self.perc_employed = perc_employed
    self.totJobs = totJobs
    self.resource_consumption = resource_consumption

    self.datacollector = mesa.DataCollector(
        {"avg_happiness":avg_happiness,
         "avg_env_cond": avg_env_cond}
    )
    # create country agents
    ac_country = mg.AgentCreator(CountryAgent, model = self, agent_kwargs ={"env_regenr": env_regen_rate})
    country_agents = ac_country.from_file(self.geojson_regions, unique_id = self.unique_id)
    self.space.add_agents(country_agents)

    ac_job = mg.AgentCreator(JobAgent, model = self, crs = self.space.crs)
    ac_human = mg.AgentCreator(HumanAgent, model = self, crs= self.space.crs, agent_kwargs={"resource_consumption": resource_consumption} )
    #create jobs
    for i in range(totJobs):
      while True:
        country = self.random.randint(0, len(country_agents)-1)
        cenx, ceny = country_agents[country].geometry.centroid.coords.xy
        bounds = country_agents[country].geometry.bounds
        spread_x = int(bounds[2] - bounds[0])
        spread_y = int(bounds[3] - bounds[1])

        x = cenx[0] + self.random.randint(0, spread_x) - spread_x / 2
        y = ceny[0] + self.random.randint(0, spread_y) - spread_y / 2
        new_job = ac_job.create_agent(Point(x, y), "J" + str(i))

        if country_agents[country].geometry.intersects(new_job.geometry):
            self.space.add_agents(new_job)
            self.schedule.add(new_job)
            break
    #fill jobs with people
    jobs = [agent for agent in self.schedule.agents if isinstance(agent, JobAgent)]
    employed = int(agents*perc_employed)
    unemployed = agents - employed

    for i in range(employed):
      x,y = jobs[i].geometry.centroid.coords.xy
      jobs[i].taken = True
      new_person = ac_human.create_agent(Point(x,y), "P" + str(i))
      self.space.add_agents(new_person)
      self.schedule.add(new_person)

    #spawn unemployed people randomly
    for i in range(unemployed):
      while True:
        country = self.random.randint(0, len(country_agents)-1)
        cenx, ceny = country_agents[country].geometry.centroid.coords.xy
        bounds = country_agents[country].geometry.bounds
        spread_x = int(bounds[2] - bounds[0])
        spread_y = int(bounds[3] - bounds[1])

        x = cenx[0] + self.random.randint(0, spread_x) - spread_x / 2
        y = ceny[0] + self.random.randint(0, spread_y) - spread_y / 2
        new_person = ac_human.create_agent(Point(x, y), "P" + str(i+employed))

        if country_agents[country].geometry.intersects(new_person.geometry):
            self.space.add_agents(new_person)
            self.schedule.add(new_person)
            break

    for agent in country_agents:
      self.schedule.add(agent)
    self.datacollector.collect(self)

  def step(self):
    self.schedule.step()
    self.datacollector.collect(self)


In [10]:
model_params = {
    "agents": mesa.visualization.Slider("Agent Number", 100, 10, 200, 10),
    "totJobs": mesa.visualization.Slider("Total Jobs", 150, 10, 300, 10),
    "perc_employed": mesa.visualization.Slider("Percent Employed", 0.5, 0.1, 1, 0.1),
    "resource_consumption": mesa.visualization.Slider("Resource Consumption", 0.1, 0.1, 1, 0.05),
    "env_regen_rate": mesa.visualization.Slider("Resource Regen Rate", 0.5, 0, 1, 0.05),
}
def portrayAgents(agent):
  portrayal = {}
  if isinstance(agent, HumanAgent):
    portrayal["radius"] = "2"
    portrayal["color"] = "Blue"
    portrayal["layer"] = "2"
  if isinstance(agent, JobAgent):
    portrayal["radius"] = "5"
    portrayal["color"] = "Black"
    portrayal["layer"] = "1"
  if isinstance(agent, CountryAgent):
    portrayal["layer"] = "0"
    if agent.env_cond > 0.4:
      portrayal["color"] = "Green"
    else:
      portrayal["color"] = "Red"
  return portrayal

map = mg.visualization.MapModule(portrayAgents)
chart = mesa.visualization.ChartModule(
    [
        {"Label": "avg_happiness", "Color": "Red"},
        {"Label": "avg_env_cond", "Color": "Green"},
    ]
)
server = mesa.visualization.ModularServer(
    EnvModel,
    [map,chart],
    "Human Environment Model",
    model_params
)

In [11]:
from pyngrok import ngrok
#pyngrok sets up tunnel to local server on web
#need because google collab is a virtual machine so cant' automatically open tab to display local servers content like mesa visualization server intends.
url = ngrok.connect(8521)
print('Open the following URL in a new tab to access the server:', url)
server.launch()

Open the following URL in a new tab to access the server: NgrokTunnel: "https://1af0-34-168-26-229.ngrok-free.app" -> "http://localhost:8521"
Interface starting at http://127.0.0.1:8521


RuntimeError: ignored